In [29]:
import os
import random
import pandas as pd
import cv2
import numpy as np

# Source dataset path
source_dir = "Dataset/Test"
# Destination directory for augmented dataset
dest_dir = "Dataset/Augmented/Test"

# Create the destination directory if it doesn't exist
os.makedirs(dest_dir, exist_ok=True)

In [3]:
train_ds = pd.read_csv("Dataset/labels_train.csv", index_col=0)
train_ds.head()

,Image path,Class,Weight
0,Dataset/Train/FBO1/cacao-31.jpg,FBO1,1.66
1,Dataset/Train/FBO1/cacao-32.jpg,FBO1,1.27
2,Dataset/Train/FBO1/cacao-33.jpg,FBO1,1.79
3,Dataset/Train/FBO1/cacao-34.jpg,FBO1,1.25
4,Dataset/Train/FBO1/cacao-35.jpg,FBO1,1.85


In [38]:
image_num=0
df = []
from tqdm import tqdm


def augment_and_save(image_path, save_dir, label, weight, image_num):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (500, 300))

    height, width = image.shape[:2] 
    
    for i in range(1, 9):

        height, width = image.shape[:2]
        center = (width // 2, height // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, 45*i, 1.0)
        rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))


        black_pixels = np.where(
            (rotated_image[:, :, 0] == 0) & 
            (rotated_image[:, :, 1] == 0) & 
            (rotated_image[:, :, 2] == 0)
        )

        # set those pixels to white
        rotated_image[black_pixels] = [210, 210, 210]

        new_file_name = f"Dataset/Augmented/Train/cacao-{image_num}.jpg"
        df.append([f"cacao-{image_num}.jpg", label, weight])

        

        cv2.imwrite(new_file_name, rotated_image)
        image_num+=1

for i in tqdm(range(len(train_ds))):
    augment_and_save(train_ds.iloc[i]["Image path"],
                     "Dataset/Augmented/Train",
                     train_ds.iloc[i]["Class"],
                     train_ds.iloc[i]["Weight"],
                     image_num)
    image_num+=8

df = pd.DataFrame(df, columns=["Image path", "Class", "Weight"])
df.to_csv("Dataset/Augmented/labels_train.csv")

100%|██████████| 1612/1612 [04:09<00:00,  6.47it/s]
